In [ ]:
# GROUP 5
# Neel Gandhi, Sunishka Jain, Daniel Shen, Julian Wu

## PREPROCESSING
### This notebook takes in the raw data from 'clinvar_conflicting.csv'
### and processes all the data, reducing our feature count to 54
### and numericizing every cell value

## OUTPUT
### A processed csv file will be created, called 'processedData.csv'

In [ ]:
# Imports

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from fractions import Fraction
from operator import itemgetter

In [ ]:
# Import data

file = './clinvar_conflicting.csv'
data = pd.read_csv(file)

In [ ]:
# This is code to create an initial correlation matrix
# We are not depicting this matrix in our paper

corr_matrix = data.corr()
# print(corr_matrix)
sns.heatmap(corr_matrix, annot=True)
plt.show()

In [ ]:
# Processing 'CHROM' column

data.loc[:, 'CHROM'].replace('X', 23, inplace=True)
data.loc[:, 'CHROM'].replace('MT', 24, inplace=True)
data.loc[:, 'CHROM'] = data.CHROM.astype(int)

In [ ]:
# Processing 'REF' column

data.loc[data['REF'] == "A", "REF"] = 1
data.loc[data['REF'] == "T", "REF"] = 2
data.loc[data['REF'] == "G", "REF"] = 3
data.loc[data['REF'] == "C", "REF"] = 4
all_other_values = data.loc[:, 'REF'].unique()
all_other_value_map = {}

for i in range(len(all_other_values)):
  all_other_value_map[all_other_values[i]] = 5 

rem_list = [1,2,3,4]
for key in rem_list:
    del all_other_value_map[key]

data['REF'] = data['REF'].replace(all_other_value_map)

# data['REF'].value_counts()

In [ ]:
# Processing 'ALT' column

data.loc[data['ALT'] == "A", "ALT"] = 1
data.loc[data['ALT'] == "T", "ALT"] = 2
data.loc[data['ALT'] == "G", "ALT"] = 3
data.loc[data['ALT'] == "C", "ALT"] = 4

all_other_values = data.loc[:, 'ALT'].unique()
all_other_value_map = {}
for i in range(len(all_other_values)):
  all_other_value_map[all_other_values[i]] = 5 

rem_list = [1,2,3,4]
for key in rem_list:
    del all_other_value_map[key]

data['ALT'] = data['ALT'].replace(all_other_value_map)

# data['ALT'].value_counts()

In [ ]:
# Processing 'CLNVC' column into one-hot encoded variables for the different variant types

data.loc[data['CLNVC'] == "single_nucleotide_variant", "CLNVC"] = 0
data.loc[data['CLNVC'] == "Deletion", "CLNVC"] = 1
data.loc[data['CLNVC'] == "Duplication", "CLNVC"] = 2
data.loc[data['CLNVC'] == "Indel", "CLNVC"] = 3
data.loc[data['CLNVC'] == "Inversion", "CLNVC"] = 4
data.loc[data['CLNVC'] == "Insertion", "CLNVC"] = 5
data.loc[data['CLNVC'] == "Microsatellite", "CLNVC"] = 6

data['CLNVC'].unique()


In [ ]:
# Processing 'IMPACT' column into dummy variables

data.loc[data['IMPACT'] == "MODERATE", "IMPACT"] = 0
data.loc[data['IMPACT'] == "MODIFIER", "IMPACT"] = 1
data.loc[data['IMPACT'] == "LOW", "IMPACT"] = 2
data.loc[data['IMPACT'] == "HIGH", "IMPACT"] = 3

data['IMPACT'] = data['IMPACT'].astype(int)

In [ ]:
# Processing 'PolyPhen' and 'SIFT' columns into dummy variables

data['PolyPhen'] = data['PolyPhen'].replace({'benign':1, 'probably_damaging':2, 'possibly_damaging' : 3})
data['PolyPhen'] = pd.to_numeric(data['PolyPhen'], errors='coerce').fillna(0, downcast='infer')
data['SIFT'] = data['SIFT'].replace({'tolerated':1, 'deleterious_low_confidence':2, 'deleterious' : 3})
data['SIFT'] = pd.to_numeric(data['SIFT'], errors='coerce').fillna(0, downcast='infer')


In [ ]:
# Processing 'Amino_Acids' column into numerical values

Amino_acid_type = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y','*','-']
Amino_acid_value_map = {}
for i in range(len(Amino_acid_type)):
  Amino_acid_value_map[Amino_acid_type[i]] = i+1

Amino_acid_ref = data['Amino_acids'].str.split("/", n=-1, expand=False).str[0]
Amino_acid_alt = data['Amino_acids'].str.split("/", n=-1, expand=False).str[1]

Amino_acid_ref = Amino_acid_ref.replace(Amino_acid_value_map)
Amino_acid_alt = Amino_acid_alt.replace(Amino_acid_value_map)

Amino_acid_ref = pd.to_numeric(Amino_acid_ref, errors='coerce').fillna(24, downcast='infer')
Amino_acid_alt = pd.to_numeric(Amino_acid_alt, errors='coerce').fillna(24, downcast='infer')


data['AA_REF'] = Amino_acid_ref 
data['AA_ALT'] = Amino_acid_alt

In [ ]:
# DataFrame for BLOSUM62 table which is symmetric diagonally

dataBLOSUM62 = [[4, 0, -2, -1, -2, 0, -2, -1, -1, -1, -1, -2, -1, -1, -1, 1, 0, 0, -3, -2], 
                [0, 9, -3, -4, -2, -3, -3, -1, -3, -1, -1, -3, -3, -3, -3, -1, -1, -1, -2, -2], 
                [-2, -3, 6, 2, -3, -1, -1, -3, -1, -4, -3, 1, -1, 0, -2, 0, -1, -3, -4, -3], 
                [-1, -4, 2, 5, -3, -2, 0, -3, 1, -3, -2, 0, -1, 2, 0, 0, -1, -2, -3, -2], 
                [-2, -2, -3, -3, 6, -3, -1, 0, -3, 0, 0, -3, -4, -3, -3, -2, -2, -1, 1, 3], 
                [0, -3, -1, -2, -3, 6, -2, -4, -2, -4, -3, 0, -2, -2, -2, 0, -2, -3, -2, -3], 
                [-2, -3, -1, 0, -1, -2, 8, -3, -1, -3, -2, 1, -2, 0, 0, -1, -2, -3, -2, 2], 
                [-1, -1, -3, -3, 0, -4, -3, 4, -3, 2, 1, -3, -3, -3, -3, -2, -1, 3, -3, -1], 
                [-1, -3, -1, 1, -3, -2, -1, -3, 5, -2, -1, 0, -1, 1, 2, 0, -1, -2, -3, -2], 
                [-1, -1, -4, -3, 0, -4, -3, 2, -2, 4, 2, -3, -3, -2, -2, -2, -1, 1, -2, -1], 
                [-1, -1, -3, -2, 0, -3, -2, 1, -1, 2, 5, -2, -2, 0, -1, -1, -1, 1, -1, -1], 
                [-2, -3, 1, 0, -3, 0, 1, -3, 0, -3, -2, 6, -2, 0, 0, 1, 0, -3, -4, -2], 
                [-1, -3, -1, -1, -4, -2, -2, -3, -1, -3, -2, -2, 7, -1, -2, -1, -1, -2, -4, -3], 
                [-1, -3, 0, 2, -3, -2, 0, -3, 1, -2, 0, 0, -1, 5, 1, 0, -1, -2, -2, -1], 
                [-1, -3, -2, 0, -3, -2, 0, -3, 2, -2, -1, 0, -2, 1, 5, -1, -1, -3, -3, -2], 
                [1, -1, 0, 0, -2, 0, -1, -2, 0, -2, -1, 1, -1, 0, -1, 4, 1, -2, -3, -2], 
                [0, -1, -1, -1, -2, -2, -2, -1, -1, -1, -1, 0, -1, -1, -1, 1, 5, 0, -2, -2], 
                [0, -1, -3, -2, -1, -3, -3, 3, -2, 1, 1, -3, -2, -2, -3, -2, 0, 4, -3, -1], 
                [-3, -2, -4, -3, 1, -2, -2, -3, -3, -2, -1, -4, -4, -2, -3, -3, -2, -3, 11, 2], 
                [-2, -2, -3, -2, 3, -3, 2, -1, -2, -1, -1, -2, -3, -1, -2, -2, -2, -1, 2, 7]]

df_BLOSUM62 = pd.DataFrame(dataBLOSUM62, 
        columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'], 
        index = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y'])

In [ ]:
# Function used for column 'ORIGIN' in order to classify
# each value into one-hot encoded origin types.
# Used in the cell below.

def get_power_list(num):
  ans = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  max_power = 9

  if num == 1073741824:
    ans[11] = 1

  elif num == 0:
    ans[0] = 1

  else:
    current_power = max_power
    while current_power >= 0:
      value = 2 ** current_power
      if num >= value:
        num -= value
        ans[current_power+1] = 1
      current_power -= 1

  return ans

In [ ]:
# Processing 'ORIGIN' column into one-hot encoded variables for each origin type

origin = pd.DataFrame()
origin['collective_list'] = data['ORIGIN'].apply(lambda x: get_power_list(x))

origin[['unknown_Origin','germline_Origin','somatic_Origin','inherited_Origin','paternal_Origin',
        'maternal_Origin','de-novo_Origin','biparental_Origin','uniparental_Origin','not-tested_Origin',
        'inconclusive_Origin','other_Origin']] = pd.DataFrame(
            origin.collective_list.tolist(), index= origin.index)

data = data.join(origin)

In [ ]:
# Function used for column 'MC' in order to classify
# each value into one-hot encoded variant types.
# Used in the cell below.

def get_variant_list(myList):
  ans = [0,0,0]

  if str(myList) == 'nan':
    return ans

  for variant in myList:
    var_str = variant.split('|')[1]
    if var_str == 'missense_variant':
      ans[0] = 1
    elif var_str == 'synonymous_variant':
      ans[1] = 1
    else:
      ans[2] = 1
  return ans

In [ ]:
# Processing 'MC' column into one-hot encoded variables
# for the top 3 different variant types

consequences = pd.DataFrame()

consequences['MC'] = data['MC'].str.split(",", n = -1, expand = False)
consequences['variantList'] = consequences['MC'].apply(lambda x: get_variant_list(x))
consequences[['missense_Variant','synonymous_Variant','other_Variant']] = pd.DataFrame(
            consequences.variantList.tolist(), index= consequences.index)

# print(consequences)
missense_col = consequences['missense_Variant']
synonymous_col = consequences['synonymous_Variant']
other_col = consequences['other_Variant']

data = data.join(missense_col)
data = data.join(synonymous_col)
data = data.join(other_col)

In [ ]:
# Processeing 'SYMBOL' column into dummy variables for the SYMBOL feature, which are just the names of the genes 

gene_names = data['SYMBOL'].unique()

#Arbitrarily assigning each of the 2329 genes to a number
for i in range(0, len(gene_names)):
    data.loc[data['SYMBOL'] == gene_names[i], "SYMBOL"] = i

In [ ]:
# Processing 'EXON' column, which shows fractions (exon #/total #)

exon_locations = data['EXON'].unique()

# Converting all the unique string forms of the fractions into floats 

for i in range(0, len(exon_locations)):
    # Pass the NaNs because we will drop rows that contain NaNs later
    if(isinstance(exon_locations[i], float)):
        pass
    else:
        data.loc[data['EXON'] == exon_locations[i], "EXON"] = float(Fraction(exon_locations[i]))
        
float_exon_locations = data['EXON'].unique()

In [ ]:
# Processing 'INTRON' column -- same concept as EXONS above

intron_locations = data['INTRON'].unique()

# Converting all the unique string forms of the fractions into floats 
for i in range(0, len(intron_locations)):
    # Pass the NaNs because we will drop rows that contain NaNs later
    if(isinstance(intron_locations[i], float)):
        pass
    else:
        data.loc[data['INTRON'] == intron_locations[i], "INTRON"] = float(Fraction(intron_locations[i]))
        
float_intron_locations = data['INTRON'].unique()


In [ ]:
print(data['INTRON'])

In [ ]:
# Using the INTRON and EXON features above, we will create two additional columns:
# 'IE': denoted as 0/1
# 'IE_Loc": location within the intron and exon

# First tackling the binary of INTRON vs. EXON as a new column
IE_binary = []
IE_location = []

for index, row in data.iterrows():
    # For some reason this condition is not being met :/
    if pd.isnull(row['EXON']):
        IE_binary.append(0)
        IE_location.append(row['INTRON'])
    # Exon
    else:
        IE_binary.append(1)
        IE_location.append(row['EXON'])

# Now adding these two additional columns into the dataframe
data['IE'] = IE_binary
data['IE_Loc'] = IE_location
data['IE'] = data['IE'].astype(int)
data['IE_Loc'] = data['IE_Loc'].astype(float)

In [ ]:
# Drop Intron and Exon columns because of the 2 new columns we created
data.drop('INTRON', inplace=True, axis=1)
data.drop('EXON', inplace=True, axis=1)


In [ ]:
# Processing 'CLNDN' column into one-hot encoded variables for each possible disease consequence

col9 = data["CLNDN"]
col9Types = {}

# Find every type of disease consequence
for s in col9:
    s = str(s)
    types = s.split('|')
    for type in types:
        if type in col9Types:
            col9Types[type] += 1
        else:
            col9Types[type] = 1

# Taking only the top N values, excluding 'not_specified' and 'not_found'
topN = 15
topNTypes = dict(sorted(col9Types.items(), key = itemgetter(1), reverse = True)[:(topN + 2)])
topNTypes.pop('not_specified')
topNTypes.pop('not_provided')

In [ ]:
# Add in the top N disease consequences to the dataframe

for type in topNTypes.keys():
    data["has_" + type] = 0

for i in range(len(col9)):
    types = str(col9[i]).split('|')
    for type in types:
        if type in topNTypes.keys():
            data.at[i, "has_" + type] = 1

In [ ]:
# Drop unnecessary columns
data = data.drop(columns=['CLNDISDB', 'CLNDISDBINCL', 'CLNDN', 'CLNDNINCL', 'CLNHGVS', 
                    'CLNSIGINCL', 'CLNVI', 'MC', 'ORIGIN', 'SSR', 'Allele',
                    'Consequence', 'Feature_type', 'Feature', 'BIOTYPE',
                    'Codons', 'DISTANCE', 'BAM_EDIT',
                    'MOTIF_NAME', 'MOTIF_POS', 'HIGH_INF_POS', 'MOTIF_SCORE_CHANGE'])

# Drop 2 more columns!
data = data.drop(columns="BLOSUM62") # dropping BLOSUM62 score FOR NOW
data = data.drop(columns='Amino_acids')

In [ ]:
# Look at missing data of the resulting columns
num_missing = data.isnull().sum()
percentage_missing = data.isnull().sum().apply(lambda x: x/data.shape[0]*100)
missing_data = pd.DataFrame({'Number of Missing':  num_missing,
                             'Percentage of Missing': percentage_missing})

missing_data['Percentage of Missing'].sort_values(ascending = False)

In [ ]:
# Drop rows with missing data
print("BEFORE dropping missing data: " + str(len(data)))
data = data.dropna()
print("AFTER dropping missing data: " + str(len(data)))

In [ ]:
# Rename columns and clean up df
data.rename(columns={"cDNA_position": "cDNA_Pos",
                    "CDS_position": "CDS_Pos",
                    "Protein_position": "Protein_Pos"               
                    })

In [ ]:
# Saved processed data to output file named 'processedData'
data.to_csv('./processedData.csv')
# print(data)
